In [ ]:
from model import SoccerPredictionDeepModel

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import copy
import json
import torch
import os
import pandas as pd

from IPython.display import display, clear_output

In [ ]:
mode = 'cnn'
model = SoccerPredictionDeepModel(mode=mode, output_size=3)

model_path = 'weights/cnn_adam/model-36.pkl'
model.load_state_dict(torch.load(model_path))

if torch.cuda.is_available():
    model.cuda()

#model.eval()

In [ ]:
with open('data/premier_league_attributes.csv', 'r') as f:
    data = json.load(f)

def get_tensor(data):
    teams = list(data.keys())
    team_to_attribute = {}
    for team in teams:
        att = torch.FloatTensor(data[team])
        team_to_attribute[team] = att
    return team_to_attribute

team_to_attribute = get_tensor(data)

In [ ]:
def create_data(teams, team_to_attribute):
    games_id = []
    games = []
    for (i, home) in enumerate(teams):
        for (j, away) in enumerate(teams):
            if home == away:
                continue
            games.append(torch.cat([team_to_attribute[home], team_to_attribute[away]], 0))
            games_id.append([i, j])
    return torch.stack(games)

In [ ]:
games = create_data(teams, team_to_attribute)

In [ ]:
def simulate_season(n, mode, model, teams, games, games_id):
    team_points = {}
    team_result = {}
    for team in teams:
        team_points[team] = 0
        team_result[team] = [0,0,0]
    for _ in range(n):
        if mode == 'cnn':
            g = games.unsqueeze(1)
            g = g.repeat(1, 3, 1, 1)
        if torch.cuda.is_available():
            g = g.cuda()
        outputs = model(g)
        _, preds = torch.max(outputs.data, 1)
        for i in range(preds.shape[0]):
            if preds[i].item() == 0:
                team_points[teams[games_id[i][0]]] += 3 / float(n)
                team_result[teams[games_id[i][0]]][0] += 1 / float(n)
                team_result[teams[games_id[i][1]]][1] += 1 / float(n)
            elif preds[i].item() == 1:
                team_points[teams[games_id[i][1]]] += 3 / float(n)
                team_result[teams[games_id[i][0]]][1] += 1 / float(n)
                team_result[teams[games_id[i][1]]][0] += 1 / float(n)
            else:
                team_points[teams[games_id[i][0]]] += 1 / float(n)
                team_points[teams[games_id[i][1]]] += 1 / float(n)
                team_result[teams[games_id[i][0]]][2] += 1 / float(n)
                team_result[teams[games_id[i][1]]][2] += 1 / float(n)
    return team_points, team_result

In [ ]:
from IPython.display import HTML, display
import tabulate

team_points, team_result = simulate_season(500, mode,  model, teams, games, games_id)
table = [[" ", "Team", "Number of Points", "W", "L", "D"],]
for (i, w) in enumerate(sorted(team_points, key=team_points.get, reverse=True)):
    table.append([i+1, w, 3*round(team_result[w][0])+round(team_result[w][2]), round(team_result[w][0]), 
                  round(team_result[w][1]), round(team_result[w][2])])

display(HTML(tabulate.tabulate(table, tablefmt='html')))

In [ ]:
player_rating = pd.read_csv('data/data.csv', encoding='latin-1')
player_rating = player_rating.dropna()
for i in player_rating['Value'].keys():
    if 'M' in player_rating['Value'][i]:
        player_rating.at[i, 'Value'] = float(player_rating['Value'][i][1:player_rating['Value'][i].index('M')]) * 10e6
    elif 'K' in player_rating['Value'][i]:
        player_rating.at[i, 'Value'] = float(player_rating['Value'][i][1:player_rating['Value'][i].index('K')]) * 10e3
    else:
        player_rating.at[i, 'Value'] = float(player_rating['Value'][i][1:])
player_rating.head()

In [ ]:
position_mapping = {'CAM':4, 'CB':2, 'CDM':4, 'CF':10, 'CM':6, 'GK':0, 'LAM':4, 'LB':3, 
                    'LCB':2, 'LCM':5, 'LDM':5, 'LF':10, 'LM':7, 'LS':9, 'LW':8, 'LWB':7,
                    'RAM':6, 'RB':1, 'RCB':2, 'RCM':6, 'RDM':6, 'RF':10, 'RM':6, 'RS':10,
                    'RW':9, 'RWB':7, 'ST':10}
positions = {'gk': ['GK'], 'def' : [['CB', 'LCB'], ['RB'], ['RCB'],['LB']], 
 'mid': [ ['CAM', 'CDM', 'LAM'], ['LCM', 'LDM'], ['CM', 'RAM', 'RCM', 'RM', 'RDM'], [ 'LM', 'LWB', 'RWB']],
 'forward': [['ST', 'RS', 'RF', 'LF', 'LS', 'CF'], ['LW','RW']]}

In [ ]:
print('Available Teams: ')
print(teams)
print()
chosen_team = input('Choose your team: ')
print()
n_players = input('How many players to recruit: ')

new_data = copy.deepcopy(data)

for i in range(int(n_players)):
    print('Available Positions:')
    for pos in positions:
        print('    ', pos, ': ', positions[pos])
    chosen_position = input('Position: ')
    player_by_position = player_rating[player_rating['Position'] == chosen_position]
    budget = input('What is your Budget? ')
    players = player_by_position[player_by_position['Value'] <= float(budget)].head()
    display(HTML(tabulate.tabulate(players, tablefmt='html')))
    chosen_player = input('Choose player index in the table (0-4): ')
    if chosen_player == 'avg':
        player = avg
    else:
        player = players.values[int(chosen_player)][3:]
    new_data[chosen_team][position_mapping[chosen_position]] = list(player)
new_team_att = get_tensor(new_data)
games = create_data(teams, team_to_attribute)
new_games = create_data(teams, new_team_att)
team_points, _ = simulate_season(500, mode, model, teams, games, games_id)
new_team_points, _ = simulate_season(500, mode, model, teams, new_games, games_id)
print('Contribution of the player is: %.2f'%(new_team_points[chosen_team] - team_points[chosen_team]))

In [ ]:
for position in position_mapping:
    best = list(player_rating[player_rating['Position'] == position].head(5).values[:][:5])
    for i in range(5):
        name = best[i][1]
        contribution = 0.0
        for t in teams:
            new_data = copy.deepcopy(data)
            new_data[t][position_mapping[position]] = list(best[i][3:])
            new_team_att = get_tensor(new_data)
            games = create_data(teams, team_to_attribute)
            new_games = create_data(teams, new_team_att)
            team_points, _ = simulate_season(20, mode, model, teams, games, games_id)
            new_team_points, _ = simulate_season(20, mode, model, teams, new_games, games_id)
            contribution += (new_team_points[chosen_team] - team_points[chosen_team]) / len(teams)
        print('Contribution of  %s %s, is: %.2f'%(position, name, contribution))

Contribution of the best CAM player, A. Griezmann, is: -0.13

Contribution of the best CB player, D. Godín, is: 0.48

Contribution of the best CDM player, Sergio Busquets, is: 0.03

Contribution of the best CF player, Luis Alberto, is: 0.30

Contribution of the best CM player, Thiago, is: -0.39

Contribution of the best GK player, De Gea, is: -0.03

Contribution of the best LAM player, J. Rodríguez, is: 0.37

Contribution of the best LB player, Marcelo, is: -0.78

Contribution of the best LCB player, G. Chiellini, is: 0.02

Contribution of the best LCM player, T. Kroos, is: 1.20

Contribution of the best LDM player, N. Kanté, is: -0.38

Contribution of the best LF player, E. Hazard, is: 0.59

Contribution of the best LM player, P. Aubameyang, is: 0.18

Contribution of the best LS player, E. Cavani, is: -0.26

Contribution of the best LW player, Neymar Jr, is: 2.07

Contribution of the best LWB player, N. Schulz, is: 0.22

Contribution of the best RAM player, J. Cuadrado, is: -0.26

Contribution of the best RB player, Azpilicueta, is: 0.18

Contribution of the best RCB player, Sergio Ramos, is: 0.62

Contribution of the best RCM player, K. De Bruyne, is: -0.87

Contribution of the best RDM player, P. Pogba, is: -0.31

Contribution of the best RF player, L. Messi, is: 0.86

Contribution of the best RM player, K. Mbappé, is: -0.40

Contribution of the best RS player, L. Suárez, is: 0.68

Contribution of the best RW player, Bernardo Silva, is: -0.19

Contribution of the best RWB player, M. Ginter, is: 0.30

Contribution of the best ST player, Cristiano Ronaldo, is: 0.54


In [ ]:
for position in position_mapping:
    worst = list(player_rating[player_rating['Position'] == position].tail(1).values[:][0])
    name = worst[1]
    contribution = 0.0
    for t in teams:
        new_data = copy.deepcopy(data)
        new_data[t][position_mapping[position]] = list(worst[3:])
        new_team_att = get_tensor(new_data)
        games = create_data(teams, team_to_attribute)
        new_games = create_data(teams, new_team_att)
        team_points, _ = simulate_season(50, mode, model, teams, games, games_id)
        new_team_points, _ = simulate_season(50, mode, model, teams, new_games, games_id)
        contribution += (new_team_points[chosen_team] - team_points[chosen_team]) / len(teams)
    print('Contribution of the best %s player, %s, is: %.2f'%(position, name, contribution))

Contribution of the best CAM player, Y. Uchimura, is: 1.51

Contribution of the best CB player, J. Williams, is: 0.64

Contribution of the best CDM player, S. Squire, is: 0.51

Contribution of the best CF player, C. Björk, is: -0.21

Contribution of the best CM player, G. Nugent, is: 1.44

Contribution of the best GK player, J. Livesey, is: 0.69

Contribution of the best LAM player, F. Sakala, is: 0.79

Contribution of the best LB player, J. Norville-Williams, is: -0.27

Contribution of the best LCB player, J. Finnerty, is: 0.26

Contribution of the best LCM player, E. Tweed, is: -1.04

Contribution of the best LDM player, Wu Lei, is: -0.75

Contribution of the best LF player, T. Matsuura, is: -0.55

Contribution of the best LM player, M. Roberts, is: 0.69

Contribution of the best LS player, G. Pender, is: 0.38

Contribution of the best LW player, Zhen'ao Wang, is: 0.22

Contribution of the best LWB player, E. Dahlqvist, is: 0.68

Contribution of the best RAM player, A. Fukuta, is: 0.32

Contribution of the best RB player, D. Walsh, is: 0.28

Contribution of the best RCB player, E. Toal, is: 0.37

Contribution of the best RCM player, Yao Junsheng, is: 0.62

Contribution of the best RDM player, B. Rivera, is: 1.09

Contribution of the best RF player, L. Grozurek, is: -0.39

Contribution of the best RM player, T. Hillman, is: 0.73

Contribution of the best RS player, J. Ellis, is: -0.16

Contribution of the best RW player, D. Walker-Rice, is: 1.11

Contribution of the best RWB player, Wang Xin, is: 0.84

Contribution of the best ST player, B. Worman, is: -0.21


In [ ]:
playas = player_rating.values[:]
pla_contrib_50 = {}
for i in range(50):
    best = list(playas[i])
    name = best[1]
    contribution = 0.0
    for t in teams:
        new_data = copy.deepcopy(data)
        new_data[t][position_mapping[best[0]]] = list(best[3:])
        new_team_att = get_tensor(new_data)
        games = create_data(teams, team_to_attribute)
        new_games = create_data(teams, new_team_att)
        team_points, _ = simulate_season(20, mode, model, teams, games, games_id)
        new_team_points, _ = simulate_season(20, mode, model, teams, new_games, games_id)
        contribution += (new_team_points[chosen_team] - team_points[chosen_team]) / len(teams)
    pla_contrib_50[name] = contribution

In [ ]:
import json
with open('contrib_50.json', 'w') as f:
    json.dump(pla_contrib_50, f)

In [ ]:
avg = [74., 78., 77., 76., 58., 51., 61., 66., 52., 60., 56., 52., 62.,
       66., 69., 69., 67., 71., 67., 64., 70., 70., 70., 56., 66., 59.,
       59., 63., 59., 52., 56., 54., 16., 18., 29., 19., 19.]

In [ ]:
for (i, w) in enumerate(sorted(pla_contrib_50, key=pla_contrib_50.get, reverse=True)):
    print(w, pla_contrib_50[w])